In [6]:
from google.colab import drive # Added

# --- Mount Google Drive ---
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Runtime: GPU (Colab > Runtime > Change runtime type > GPU)
!nvidia-smi
!python --version

# Install once
!pip -q install thop pandas


Sat Nov  8 13:03:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install ptflops


In [4]:
import torch
import torch.nn as nn

In [5]:


def conv(in_channels, out_channels, kernel_size, bias=False, padding = 1, stride = 1):
    return nn.Conv2d(in_channels, out_channels, kernel_size,
        padding=(kernel_size//2), bias=bias, stride=stride)

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=False, bias=False):
        super().__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes, eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None
    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None: x = self.bn(x)
        if self.relu is not None: x = self.relu(x)
        return x

class ChannelPool(nn.Module):
    def forward(self, x):
        import torch
        return torch.cat((torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1)

class spatial_attn_layer(nn.Module):
    def __init__(self, kernel_size=5):
        super().__init__()
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1)//2, relu=False)
    def forward(self, x):
        import torch
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = torch.sigmoid(x_out)
        return x * scale

class ca_layer(nn.Module):
    def __init__(self, channel, reduction=8, bias=True):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_du = nn.Sequential(
            nn.Conv2d(channel, channel // reduction, 1, padding=0, bias=bias),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel // reduction, channel, 1, padding=0, bias=bias),
            nn.Sigmoid()
        )
    def forward(self, x):
        y = self.avg_pool(x)
        y = self.conv_du(y)
        return x * y

class SCA(nn.Module):
    def __init__(self, n_feat, kernel_size=3, reduction=8, bias=False, bn=False, act=nn.PReLU(), res_scale=1):
        super().__init__()
        self.body = nn.Sequential(conv(n_feat, n_feat, kernel_size, bias=bias), act, conv(n_feat, n_feat, kernel_size, bias=bias))
        self.SA = spatial_attn_layer()
        self.CA = ca_layer(n_feat, reduction, bias=bias)
        self.conv1x1 = nn.Conv2d(n_feat*2, n_feat, kernel_size=1, bias=bias)
    def forward(self, x):
        import torch
        res = self.body(x)
        sa_branch = self.SA(res)
        ca_branch = self.CA(res)
        res = torch.cat([sa_branch, ca_branch], dim=1)
        res = self.conv1x1(res)
        res += x
        return res

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
        )
    def forward(self, x): return x + self.conv_block(x)

class UNetDown(nn.Module):
    def __init__(self, in_size, out_size, bn=True):
        super().__init__()
        layers = [nn.Conv2d(in_size, out_size, 4, 2, 1, bias=False)]
        if bn: layers.append(nn.BatchNorm2d(out_size, momentum=0.8))
        layers.append(nn.LeakyReLU(0.2))
        layers.append(ResidualBlock(out_size, out_size))
        self.model = nn.Sequential(*layers)
    def forward(self, x): return self.model(x)

class UNetUp(nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        layers = [
            nn.ConvTranspose2d(in_size, out_size, 4, 2, 1, bias=False),
            nn.BatchNorm2d(out_size, momentum=0.8),
            nn.ReLU(inplace=True),
        ]
        layers.append(ResidualBlock(out_size, out_size))
        self.model = nn.Sequential(*layers)
    def forward(self, x, skip_input):
        import torch
        x = self.model(x)
        x = torch.cat((x, skip_input), 1)
        return x

class MuLA_GAN_Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()
        self.down1 = UNetDown(in_channels, 32, bn=False)
        self.down2 = UNetDown(32, 128)
        self.down3 = UNetDown(128, 256)
        self.down4 = UNetDown(256, 256)
        self.down5 = UNetDown(256, 256, bn=False)
        self.attention1 = SCA(32)
        self.attention2 = SCA(128)
        self.attention3 = SCA(256)
        self.attention4 = SCA(256)
        self.up1 = UNetUp(256, 256)
        self.up2 = UNetUp(512, 256)
        self.up3 = UNetUp(512, 128)
        self.up4 = UNetUp(256, 32)
        self.final = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(64, out_channels, 4, padding=1),
            nn.Tanh(),
        )
    def forward(self, x):
        d1 = self.down1(x); d1 = self.attention1(d1)
        d2 = self.down2(d1); d2 = self.attention2(d2)
        d3 = self.down3(d2); d3 = self.attention3(d3)
        d4 = self.down4(d3); d4 = self.attention4(d4)
        d5 = self.down5(d4)
        u1 = self.up1(d5, d4)
        u2 = self.up2(u1, d3)
        u3 = self.up3(u2, d2)
        u4 = self.up4(u3, d1)
        return self.final(u4)


In [8]:
model = MuLA_GAN_Generator().eval()

ckpt_path = "/content/drive/MyDrive/EUVP_merged/generator_300_UFO120.pth"  # <--- your path
state = torch.load(ckpt_path, map_location="cpu")

# Remove 'module.' prefix if it exists
state = {k.replace("module.", ""): v for k,v in state.items()}

model.load_state_dict(state, strict=True)
print("✅ Weights loaded")


✅ Weights loaded


In [9]:
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device).eval()

x = torch.randn(1, 3, 256, 256).to(device)

# warm-up
for _ in range(10):
    _ = model(x)
torch.cuda.synchronize() if device.type == "cuda" else None

# measure
start = time.time()
for _ in range(50):
    _ = model(x)
torch.cuda.synchronize() if device.type == "cuda" else None
end = time.time()

lat = (end-start)/50
print(f"⏱ Latency: {lat*1000:.2f} ms/image")
print(f"⚡ Throughput: {1/lat:.2f} images/sec")


⏱ Latency: 9.46 ms/image
⚡ Throughput: 105.67 images/sec


In [10]:
import copy

def replace_prelu_with_relu(m):
    import torch.nn as nn
    for name, module in m.named_children():
        if isinstance(module, nn.PReLU):
            setattr(m, name, nn.ReLU())
        else:
            replace_prelu_with_relu(module)
    return m

model_for_flops = replace_prelu_with_relu(copy.deepcopy(model.cpu())).eval()


In [11]:
from ptflops import get_model_complexity_info

with torch.no_grad():
    flops, params = get_model_complexity_info(
        model_for_flops,
        (3, 256, 256),
        verbose=False
    )

print("📌 Parameters:", params)
print("📌 FLOPs:", flops)


MuLA_GAN_Generator(
  16.56 M, 100.000% Params, 13.09 GMac, 99.915% MACs, 
  (down1): UNetDown(
    20.16 k, 0.122% Params, 331.35 MMac, 2.530% MACs, 
    (model): Sequential(
      20.16 k, 0.122% Params, 331.35 MMac, 2.530% MACs, 
      (0): Conv2d(1.54 k, 0.009% Params, 25.17 MMac, 0.192% MACs, 3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(0, 0.000% Params, 524.29 KMac, 0.004% MACs, negative_slope=0.2)
      (2): ResidualBlock(
        18.62 k, 0.112% Params, 305.66 MMac, 2.334% MACs, 
        (conv_block): Sequential(
          18.62 k, 0.112% Params, 305.66 MMac, 2.334% MACs, 
          (0): Conv2d(9.25 k, 0.056% Params, 151.52 MMac, 1.157% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, 0.000% Params, 1.05 MMac, 0.008% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(0, 0.000% Params, 524.29 KMac, 0.004% MACs, inplace=True)
          (3): Conv2